# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
###########
# Study data files
path = '../output_data/weather_data.csv'

# Read the weather data and the study results
weather_data = pd.read_csv(path)

# Combine the data into a single dataset
df = pd.DataFrame(weather_data)
df.head()




,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temp
0,0,morehead,37.27,-87.18,89.60,59,1,4.70,US,08/10/20,88.77
1,1,busselton,-33.65,115.33,55.99,78,98,19.86,AU,08/10/20,54.03
2,2,ponta do sol,32.67,-17.10,77.00,65,40,14.99,PT,08/10/20,72.68
3,3,kailua,21.40,-157.74,77.00,69,40,10.29,US,08/10/20,75.83
4,4,lebu,-37.62,-73.65,48.88,60,28,16.37,CL,08/10/20,48.88


In [44]:
#####################
# Confirm lengths are the same
print(f"lat: {len(df['Latitude'])}")
print(f"lng: {len(df['Longitude'])}")
print(f"max_temp: {len(df['Max Temp'])}")
print(f"humidity: {len(df['Humidity'])}")
print(f"cloudiness: {len(df['Cloudiness'])}")
print(f"wind_speed: {len(df['Wind Speed'])}")
print(f"country: {len(df['Country'])}")
print(f"date: {len(df['Date'])}")
print(f"temp: {len(df['Temp'])}")

lat: 554
lng: 554
max_temp: 554
humidity: 554
cloudiness: 554
wind_speed: 554
country: 554
date: 554
temp: 554


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = df[['Latitude', 'Longitude']].astype(float)

df = df.dropna()
df
hum_rate = df['Humidity'].astype(float)



In [57]:
# Create a Heatmap layer
fig = gmaps.figure(center=(40, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
plt.savefig('../output_data/vac_heatmap.png')

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# Narrow down the DataFrame to find your ideal weather condition.

# Filter for a max temp below 80 degrees but higher than 70.
below80andabove70 = df.loc[(df['Max Temp']<80) & (df['Max Temp']>70),:]
below80andabove70.head()

# Filter for Wind speed below 10 mph.
WSabove10 = below80andabove70.loc[below80andabove70['Wind Speed']<10,:]
WSabove10

# Filter for cloudiness equals zero
noclouds = WSabove10.loc[WSabove10['Cloudiness']==0,:]
noclouds.head(10)

# Drop any rows with any of these three conditions returning missing 
# or blank reuslts
narrowed_df = noclouds.dropna(how='any')
narrowed_df.reset_index(inplace=True)
del narrowed_df['index']
narrowed_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temp
0,7,sinazongwe,-17.26,27.46,73.18,37,0,3.09,ZM,08/10/20,73.18
1,30,changli,39.70,119.15,72.45,87,0,0.96,CN,08/10/20,72.45
2,86,kigoma,-4.88,29.63,75.69,60,0,3.96,TZ,08/10/20,75.69
3,87,castro,-24.79,-50.01,73.51,44,0,8.63,BR,08/10/20,73.51
4,114,cidreira,-30.18,-50.21,77.36,47,0,7.52,BR,08/10/20,77.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_list = []

for i in range(len(narrowed_df)):
    lat = narrowed_df.loc[i]['Latitude']
    lng = narrowed_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_list.append("")
narrowed_df["Hotel Name"] = hotel_list
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.head()



,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temp,Hotel Name
0,7,sinazongwe,-17.26,27.46,73.18,37,0,3.09,ZM,08/10/20,73.18,Sinazongwe
1,30,changli,39.70,119.15,72.45,87,0,0.96,CN,08/10/20,72.45,Qinhuangdao
2,86,kigoma,-4.88,29.63,75.69,60,0,3.96,TZ,08/10/20,75.69,Kigoma
3,87,castro,-24.79,-50.01,73.51,44,0,8.63,BR,08/10/20,73.51,Castro
4,114,cidreira,-30.18,-50.21,77.36,47,0,7.52,BR,08/10/20,77.36,CIDREIRA


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_df.iterrows()]
locations = narrowed_df[["Latitude", "Longitude"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))